In [30]:
import qiskit
import math
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.aqua.components.optimizers import COBYLA,SPSA
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.test.mock import FakeVigo
from qiskit.tools.monitor import job_monitor
from scipy.optimize import minimize

In [31]:
def create_circuit(parameters):
    
    qctl = QuantumRegister(2)
    qc = ClassicalRegister(2)
    circ = QuantumCircuit(qctl, qc)

    circ.rx(parameters[0],qctl[0])
    circ.ry(parameters[1],qctl[0])
    circ.rx(parameters[2],qctl[1])
    circ.cx(qctl[0],qctl[1])
    
    circ.measure(qctl,qc)
    
    return circ

In [32]:
def calculate_cost_function(parameters):
    
    device_backend = FakeVigo()
    coupling_map = device_backend.configuration().coupling_map
    
    phase=0
    probability={}
    times=1000
    
    simulator = QasmSimulator()
    
    #p_gate1 = 0.005
    #p_meas = 0.005
    
    circ = create_circuit(parameters)
    
    #error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    #error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    #error_gate2 = error_gate1.tensor(error_gate1)
    
    #noise_model= NoiseModel()
    
    #noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    #noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])
    #noise_model.add_all_qubit_quantum_error(error_gate1, ["ry", "rx"])
    
    noise_model = NoiseModel.from_backend(device_backend)
    job = execute(circ, simulator,
                basis_gates = noise_model.basis_gates,
                noise_model = noise_model,
                shots = times)    
    counts = job.result().get_counts()
    
    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0
        
    for i in counts.keys():
        probability[i] = counts[i]/times
    
    rev_phase= (np.cos(parameters[0]/2)*np.sin(parameters[1]/2) - 1j*np.sin(parameters[0]/2)*np.cos(parameters[1]/2)) / (np.cos(parameters[0]/2)*np.cos(parameters[1]/2) + 1j*np.sin(parameters[0]/2)*np.sin(parameters[1]/2))
    
    print (probability,rev_phase)
    return ((probability['00'])**2 + (probability['01']-1/2)**2 + (probability['10']-1/2)**2 + (probability['11'])**2)+ (abs(rev_phase-np.exp(1j*phase)))**2  

In [33]:
outcome = minimize(calculate_cost_function, x0=[0,0,0], method="POWELL", options={'maxiter':500})
print(outcome)

{'00': 0.978, '01': 0.01, '10': 0.012, '11': 0.0} 0j
{'00': 0.984, '01': 0.013, '10': 0.003, '11': 0.0} 0j
{'00': 0.737, '01': 0.015, '10': 0.012, '11': 0.236} -0.5463024898437905j
{'00': 0.058, '01': 0.031, '10': 0.021, '11': 0.89} -3.7323503047403657j
{'00': 0.733, '01': 0.011, '10': 0.012, '11': 0.244} -0.5463024898437905j
{'00': 0.449, '01': 0.015, '10': 0.015, '11': 0.521} -1.048389550684371j
{'00': 0.898, '01': 0.009, '10': 0.013, '11': 0.08} -0.31924398056579173j
{'00': 0.842, '01': 0.003, '10': 0.008, '11': 0.147} -0.442099088358064j
{'00': 0.668, '01': 0.018, '10': 0.013, '11': 0.301} -0.7109451330728335j
{'00': 0.721, '01': 0.015, '10': 0.015, '11': 0.249} -0.6063628940833873j
{'00': 0.771, '01': 0.011, '10': 0.01, '11': 0.208} -0.5054721572208577j
{'00': 0.763, '01': 0.014, '10': 0.006, '11': 0.217} -0.522606700071272j
{'00': 0.797, '01': 0.01, '10': 0.009, '11': 0.184} -0.4808965540807323j
{'00': 0.785, '01': 0.016, '10': 0.006, '11': 0.193} -0.49602840421196864j
{'00': 0.7

In [34]:
help(minimize)

Help on function minimize in module scipy.optimize._minimize:

minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
    Minimization of scalar function of one or more variables.
    
    Parameters
    ----------
    fun : callable
        The objective function to be minimized.
    
            ``fun(x, *args) -> float``
    
        where ``x`` is an 1-D array with shape (n,) and ``args``
        is a tuple of the fixed parameters needed to completely
        specify the function.
    x0 : ndarray, shape (n,)
        Initial guess. Array of real elements of size (n,),
        where 'n' is the number of independent variables.
    args : tuple, optional
        Extra arguments passed to the objective function and its
        derivatives (`fun`, `jac` and `hess` functions).
    method : str or callable, optional
        Type of solver.  Should be one of
    
            - 'Nelder-Mead' :ref:`(see her